# Iterative Prompt Develelopment

Goal of this notebook is practicing and understanding iterative prompt development principles.

Examples sourced from [Deeplearning.ai ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/3/iterative) course.

LLM Model: [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b)

In [1]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1
import time

import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# print(torch.backends.mps.is_available())
# print(torch.backends.mps.is_built())
# print(torch.cuda.is_available())

In [2]:
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

simple_prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=simple_prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)


def get_response(input_prompt, context=None):
    if context is None:
        return llm_chain.predict(instruction=input_prompt).lstrip()
    else:
        return llm_context_chain.predict(instruction=input_prompt, context=context).lstrip()


def reply_prompt(input_prompt, context=None):
    start = time.time()
    response = get_response(input_prompt, context)
    end = time.time()
    duration_seconds = end - start
    print(response)
    print(f"\nProcessing duration: {duration_seconds:0.2f} seconds")


## Challenge: Generate a marketing product description from a product fact sheet

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [9]:
prompt = f"""
Your task is to create a description for a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by <>

Technical specifications: <{fact_sheet_chair}>
"""
reply_prompt(prompt)

The product described is a beautiful mid-century office furniture called SWC-110 with stainless steel base and various finish options. This furniture includes: 
- filing cabinets,
- desks,
- bookcases, and 
- meeting tables.

Processing duration: 303.47 seconds


### Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [10]:
prompt = f"""
Your task is to create a description for a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by <>

Use at most 200 characters.

Technical specifications: <{fact_sheet_chair}>
"""
reply_prompt(prompt)

Part of a beautiful family of mid-century inspired office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, the SWC chairs feature pneumatic chair adjust for easy raise/lower action, and base finishes of stainless steel, chrome, or matte black. Two options of soft or hard floor castors, 8 position PU armrests, and two choices of seat foam density are available for the SWC chairs.

Processing duration: 338.34 seconds


### Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [11]:
prompt = f"""
Your task is to create a description for a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by <>.

The description is intended for furniture retailers, so should be technical in nature and focus on the materials the product is constructed from.

Use at most 200 characters.

Technical specifications: <{fact_sheet_chair}>
"""

reply_prompt(prompt)

This office furniture product is from the mid-century modern style and is part of a beautiful family of products including filing cabinets, desks, bookcases, meeting tables, and more.

Several options of shell color and base finishes are available.

Base finish options are stainless steel, matte black, gloss white, or chrome.

The chair in this product is available with or without armrests. 

There are soft or hard floor caster options.

Two choices of seat foam density are available: medium (1.8 lbs/ft3) or high (2.8 lbs/ft3).

PU armrests are available in 8 positions.

This product can be used for home or business use.

Processing duration: 400.53 seconds


In [12]:
prompt = f"""
Your task is to create a description for a product based on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by <>.

Use at most 200 characters.

The description should be technical in nature and focus on the materials the product is constructed from.

Technical specifications: <{fact_sheet_chair}>
"""

reply_prompt(prompt)

Here is an example of a technical specifications for a product:
- Part of a beautiful family of mid-century inspired office furniture, 
 - including filing cabinets, desks, bookcases, meeting tables, and more.
 - Available with plastic back and front upholstery (SWC-100) 
 - or full upholstery (SWC-110) in 10 fabric and 6 leather options.
 - Stainless steel base finish options are: 
 - stainless steel, matte black, 
 - gloss white, or chrome.
 - Chair is available with or without armrests.
 - Suitable for home or business settings.
 - Qualified for contract use.

Processing duration: 388.89 seconds
